In [2]:
from Model.model import ModelSTGCN
import torch
from torch.utils.data import DataLoader,Dataset
import pandas as pds
import numpy as np
import torch.nn as nn

c:\Users\ADMIN\anaconda3\envs\pythonProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class ActionDataset(Dataset):
    def __init__(self,Transform=None):
        super().__init__()
        # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        self.transform = Transform
        self.lst=[]
        self.label=[]
        # self.append(Data,label)

    def __getitem__(self, index):
        sample = self.lst[index],self.label[index]
        if self.transform:
            sample=self.transform(sample)
        return sample
    
    def __len__(self):
        return self.leng
    
    def SetTrans(self,Transform):
        self.transform=Transform
    
    def append(self,Data):
        # super().__init__()
        # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        # assert label==None,"Label must be give"
        kpscore = np.expand_dims(Data['kp_score'][0],axis=2)
        kp = Data['kp'][0]
        data = np.concatenate((kp,kpscore),axis=2)
        data = np.expand_dims(data,axis=0)
        label = Data['label']
        self.lst.append(data)
        self.label.append(label)
        self.leng=len(self.lst)  

class ToTensor():
    def __call__(self,sample):
        data,label=sample
        return torch.from_numpy(data),torch.tensor(label)

In [ ]:
def Train_val(model,dataloader,epochs,lr):
    device=torch.device('cuda')
    model.to(device)
    optim=torch.optim.adam(model.parameters(),lr)
    criteria=nn.CrossEntropyLoss()
    for epoch in range(epochs):
        for i,(data,label) in enumerate (dataloader):
            predict=model(data.to(device))
            label=label.to(device)
            loss=criteria(label,predict)
            optim.zero_grad()
            loss.backward()
            optim.step()
        print(f'Epoch: {epoch}, step: {i}/{len(dataloader)}')

In [6]:
datalst=pds.read_pickle('train.pkl')
datalst


[{'kp': array([[[[0.4822971 , 0.27934727],
           [0.5013769 , 0.26662734],
           [0.46321732, 0.26662734],
           ...,
           [0.43459755, 0.66094345],
           [0.50614685, 0.8072221 ],
           [0.44890743, 0.72454286]],
  
          [[0.48238975, 0.27949473],
           [0.50149983, 0.26675466],
           [0.46327972, 0.26675466],
           ...,
           [0.43461466, 0.6553258 ],
           [0.50627726, 0.8082062 ],
           [0.4441696 , 0.725396  ]],
  
          [[0.4823212 , 0.2797617 ],
           [0.50144017, 0.26701573],
           [0.46320224, 0.26701573],
           ...,
           [0.43452382, 0.65576804],
           [0.5062199 , 0.80871975],
           [0.44408327, 0.7194979 ]],
  
          ...,
  
          [[0.46772614, 0.3093913 ],
           [0.48334527, 0.29550755],
           [0.4469006 , 0.29550755],
           ...,
           [0.43128142, 0.6981347 ],
           [0.43128142, 0.725902  ],
           [0.4469006 , 0.8439135 ]],
  
        

In [7]:
actDataset=ActionDataset(ToTensor())
for data in datalst:
    actDataset.append(data)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [8]:
data

{'kp': array([], shape=(0, 33, 17, 2), dtype=float32),
 'kp_score': array([], shape=(0, 33, 17), dtype=float32),
 'frame_dir': 'VIDEO1_4',
 'img_shape': (640, 480),
 'original_shape': (640, 480),
 'total_frames': 33,
 'label': 2}

In [4]:
batch_size=4
datalst=pds.read_pickle('train.pkl')
actDataset=ActionDataset(datalst[0])
dataloader=DataLoader(dataset=actDataset,batch_size=batch_size,shuffle=True)
n_sample=len(actDataset)
epochs=10
n_iter=round(n_sample/batch_size)
for epoch in range(epochs-8):
    for i,data in enumerate (dataloader):
        if (i+1)% (batch_size)==0:
            print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}, Input shap: {data.shape}')

AttributeError: 'ActionDataset' object has no attribute 'leng'

In [ ]:
import math
math.ceil(len(actDataset)/batch_size)

In [ ]:
batch_size=4
datalst=pds.read_pickle('train.pkl')
actDataset=ActionDataset(datalst[0])
dataloader=DataLoader(dataset=actDataset,batch_size=batch_size,shuffle=True)
n_sample=len(actDataset)
n_iter=round(n_sample/batch_size)
model = ModelSTGCN(3,5)
lr = 1e-3
optim = torch.optim.Adam(model.parameters(),lr)
lr_schedule = torch.optim.lr_scheduler.MultiStepLR(optim,[15,30],gamma=0.1)
epochs = 10
for epoch in range(epochs):
    for i,data in enumerate (dataloader):
        if (i+1)% (batch_size)==0:
            print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}')